In [1]:
# Import data into a pandas dataframe 
import pandas as pd 
yeast_dataset = pd.read_csv("yeast.data", header=None)
yeast_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1484 entries, 0 to 1483
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       1484 non-null   object 
 1   1       1484 non-null   float64
 2   2       1484 non-null   float64
 3   3       1484 non-null   float64
 4   4       1484 non-null   float64
 5   5       1484 non-null   float64
 6   6       1484 non-null   float64
 7   7       1484 non-null   float64
 8   8       1484 non-null   float64
 9   9       1484 non-null   object 
dtypes: float64(8), object(2)
memory usage: 116.1+ KB


In [2]:
yeast_dataset.iloc[:,-1] = yeast_dataset.iloc[:,-1].apply(lambda x : {
    "CYT" : 0,
    "NUC" : 1,
    "MIT" : 2,
    "ME3" : 3,
    "ME2" : 4,
    "ME1" : 5, 
    "EXC" : 6,
    "VAC" : 7,
    "POX" : 8,
    "ERL" : 9
}[x])

In [3]:
yeast_dataset = yeast_dataset.iloc[:,1:]
yeast_dataset.head()

,1,2,3,4,5,6,7,8,9
0,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,2
1,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,2
2,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,2
3,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,1
4,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,2


In [4]:
# Split data to train (80%) and test (20%) data 
from sklearn.model_selection import train_test_split
X = yeast_dataset.iloc[:,:-1]
y = yeast_dataset.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [5]:
# Import saved models 
import pickle
clf_autosklearn = pickle.load(open('autosklearn_yeast.sav', 'rb'))
clf_oboe = pickle.load(open('oboe_yeast.sav', 'rb'))

In [6]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from tpot.builtins import StackingEstimator



# Average CV score on the training set was: 0.6108321809736552
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=RandomForestClassifier(bootstrap=True, criterion="entropy", max_features=0.35000000000000003, min_samples_leaf=10, min_samples_split=20, n_estimators=100)),
    MultinomialNB(alpha=0.001, fit_prior=False)
)

exported_pipeline.fit(X_train, y_train)

# Get predictions for test datasets 
tpot_pred = exported_pipeline.predict(X_test)
sklearn_pred = clf_autosklearn.predict(X_test)
oboe_pred = clf_oboe.predict(X_test)[0]

/home/datagenius/anaconda3/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [8]:
from sklearn.metrics import accuracy_score

# Accuracy for each tool 
print(accuracy_score(y_test,sklearn_pred ))
print(accuracy_score(y_test,oboe_pred ))
print(accuracy_score(y_test,tpot_pred ))

# The results are again different because we have a diffirent random split :/ 
# TPOT score : 		  0.6363636363636364
# OBOE score : 		  0.7070707070707071
# AutoSklearn: 		  0.6430976430976431

0.5757575757575758
0.7070707070707071
0.5656565656565656


In [9]:
# Ensemble learning Stacking 
# This is not the way it has to be (I loaded the models that were trained on a diffirent dataset != X_train)
# Get predictions of each tool on the train dataset 
tpot_pred    = exported_pipeline.predict(X_train)
sklearn_pred = clf_autosklearn.predict(X_train)
oboe_pred    = clf_oboe.predict(X_train)[0]

In [10]:
from sklearn.metrics import accuracy_score

# Accuracy for each tool (Again this is nt how it is supposed to be done !!)
print(accuracy_score(y_train,sklearn_pred ))
print(accuracy_score(y_train,oboe_pred ))
print(accuracy_score(y_train,tpot_pred ))

0.6217354675652906
0.7135636057287279
0.6849199663016007


In [11]:
# Create training data for metalearner (the predictions on the train dataset is used as train data for metalearner)
prob = pd.DataFrame(list(zip(sklearn_pred, oboe_pred, tpot_pred, y_train)), columns=[ "AutoSklearn_Class", "Oboe_Class", "TPOT_Class", "expetected"])
X = prob.drop(columns = ['expetected'])
Y = prob['expetected']

In [12]:
X

,AutoSklearn_Class,Oboe_Class,TPOT_Class
0,2,2,2
1,6,4,4
2,1,1,1
3,2,2,2
4,1,1,1
...,...,...,...
1182,1,1,1
1183,0,1,1
1184,2,2,2
1185,0,0,0


In [13]:
from xgboost import XGBClassifier

clf = XGBClassifier()
clf.fit(X, Y)

/home/datagenius/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:22:35] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=12, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [14]:
# The accuracy of metalearner on the data used for training (already seen data)
print(accuracy_score(Y,clf.predict(X) ))

0.7430497051390059


In [15]:
# Construct the test data 
tpot_pred_test    = exported_pipeline.predict(X_test)
sklearn_pred_test = clf_autosklearn.predict(X_test)
oboe_pred_test    = clf_oboe.predict(X_test)[0]

prob_test = pd.DataFrame(list(zip(sklearn_pred_test, oboe_pred_test, tpot_pred_test)), columns=[ "AutoSklearn_Class", "Oboe_Class", "TPOT_Class"])


In [17]:
# The accuracy of metalearner  
ensemble = clf.predict(prob_test)
print(accuracy_score(y_test,ensemble ))

0.632996632996633


In [10]:
# Voting ensemble Learning
  
tpot_sklearn_prob = clf_tpot.predict_proba(X_test)
auto_sklearn_prob = clf_autosklearn.predict_proba(X_test)
AutoSklearn_Class=[]
AutoSklearn_Prob= []
TPOT_Class=[]
TPOT_Prob= []
for  value in auto_sklearn_prob :
    AutoSklearn_Class.append(int(np.argmax(value)))
    AutoSklearn_Prob.append(float(value[np.argmax(value)]))

for  value in tpot_sklearn_prob :
    TPOT_Class.append(int(np.argmax(value)))
    TPOT_Prob.append(float(value[np.argmax(value)])) 

prob = pd.DataFrame(list(zip(AutoSklearn_Class, AutoSklearn_Prob, TPOT_Class, TPOT_Prob)), columns=[ "AutoSklearn_Class", "AutoSklearn_Prob", "TPOT_Class", "TPOT_Prob"])
prob['ensemble'] = prob.apply(lambda x : "T" if x['AutoSklearn_Prob']<= x['TPOT_Prob'] else "A", axis=1)
prob.head()

,AutoSklearn_Class,AutoSklearn_Prob,TPOT_Class,TPOT_Prob,ensemble
0,1,0.197171,1,0.292026,T
1,0,0.162204,0,0.365423,T
2,0,0.194128,0,0.381096,T
3,0,0.174253,0,0.363878,T
4,0,0.167299,0,0.351412,T


In [12]:
prob["ensemble"].value_counts()

T    295
A      2
Name: ensemble, dtype: int64

In [13]:
prob['pred'] = prob.apply(lambda x : x['TPOT_Class'] if x['ensemble'] == 'T' else x['AutoSklearn_Class'], axis=1)
accuracy_score(y_test, prob['pred'])

0.6430976430976431